In [17]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Part II: Model Building

Here you try your hand at model building to predict appointment no shows.

### Preprocessing

Package 'proj2_lib' now includes code to carry out preprocessing steps from part I. Here's how to use it:

In [18]:
import proj2_lib.util as utils

First, it includes a dictionary used for configuring path and file names
used through the project

In [19]:
utils.file_config

{'feature_pipeline_file': 'feature_pipeline.pkl',
 'labels_pipeline_file': 'labels_pipeline.pkl',
 'objstore_path': 'objects',
 'processed_data_path': 'processed_data',
 'raw_data_csv': 'KaggleV2-May-2016.csv',
 'raw_data_path': 'data',
 'test_csv': 'test_set.csv',
 'train_csv': 'train_set.csv'}

`feature_pipeline_file`: file storing the preprocessing pipeline used for preparing the feature matrix

`labels_pipeline_file`: file storing the preprocessing pipeline used for
preparing labels

`objstore_path`: directory to store python objects to disk

`processed_data_path`: directory containing processed data

`raw_data_csv`: name of the csv download from Kaggle

`raw_data_path`: directory containing raw data

`test_csv`: name of csv file containing test set

`train_csv`: name of csv file containing train set

You can change these paths and names to suit your project directory structure if you need so. E.g.,

In [20]:
file_config = utils.file_config
#config['raw_data_path'] = "some_other_directory"

First step is to create train test sets. Code is in file `proj2_lib/util.py` function `make_train_test_sets`. You
can edit that function as needed to include your own part I code if you so desire. The result will be to 
create files `train_set.csv` and `test_set.csv` in your `processed_data` directory (unless you change any of the entries in the configuration directory as above).

In [22]:
# ONLY NEED TO RUN THIS STEP ONCE (switch this to True to run it)
RUN_MAKE_TRAIN_TEST_FILES = False
if RUN_MAKE_TRAIN_TEST_FILES:
    utils.make_train_test_sets(config=file_config)

Next step is to fit the preprocessing pipelines. This is done in file `proj2_lib/preprocess.py`. Again you can edit code as needed in that file to incorporate your part I solution as you wish. The result will be to create files `feature_pipeline.pkl` and `labels_pipeline.pkl` containing the fit preprocessing pipelines we can then use to preprocess data.

In [23]:
import proj2_lib.preprocess as preprocess

# ONLY NEED TO RUN THIS STEP ONCE
RUN_FIT_PREPROCESSING = False
if RUN_FIT_PREPROCESSING:
    preprocess.fit_save_pipelines(config=file_config)

Finally, once we do that, we can get a training matrix and labels:

In [24]:
train_X, train_y = preprocess.load_train_data(config=file_config)

In [25]:
print(train_X.shape)
print(train_y.shape)

(90526, 101)
(90526,)


### Model Building

Using `sklearn` fit:
    - DecisionTree classifier
    - RandomForest classifier
    - Linear SVM classifier
    - SVM with Radial Basis Kernel classifier
    
Use default parameters for now.
Using 10-fold cross validation report both accuracy and AUC for each of the above four models.

QUESTION: Should you use accuracy or AUC for this task as a performance metric?

_Since the class distribution is skewed it is better to use AUC as a measure of performance_.

In [26]:
# build your models here

### Decision Tree classifier

In [27]:
from sklearn.tree import DecisionTreeClassifier

tree_classifier = DecisionTreeClassifier()
tree_classifier.fit(train_X, train_y)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [28]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(tree_classifier, train_X, train_y, 
                         scoring='accuracy', cv=10)

In [29]:
# mean accuracy of Decision Tree classifier
import numpy as np
np.mean(scores)

0.73815249315673759

In [30]:
auc_scores = cross_val_score(tree_classifier, train_X, train_y,
                            scoring='roc_auc', cv=10)

In [31]:
# mean AUC score of Decision Tree classifier
np.mean(auc_scores)

0.58206403316070243

### Random Forest Classifier

In [32]:
from sklearn.ensemble import RandomForestClassifier

rf_classifier = RandomForestClassifier()
rf_accuracy = cross_val_score(rf_classifier, train_X, train_y, 
                             scoring='accuracy', cv=10)

In [33]:
# mean accuracy of Random Forest classifier
np.mean(rf_accuracy)

0.77619678688944216

In [34]:
rf_auc = cross_val_score(rf_classifier, train_X, train_y, 
                        scoring='roc_auc', cv=10)

In [35]:
# mean AUC of RF classifier
np.mean(rf_auc)

0.69270382869394798

### Linear SVM

In [36]:
from sklearn.svm import LinearSVC

linear_svm = LinearSVC(dual=False)
lsvm_accuracy = cross_val_score(linear_svm, train_X, train_y,
                               scoring='accuracy', cv=10)

In [37]:
# mean accuracy of linear SVM
np.mean(lsvm_accuracy)

0.79625743992306519

In [38]:
lsvm_auc = cross_val_score(linear_svm, train_X, train_y,
                          scoring='roc_auc', cv=10)

In [39]:
# mean AUC of linear SVM
np.mean(lsvm_auc)

0.6673264347754625

### Non-linear SVM

In [40]:
# not running this to save some time
from sklearn.svm import SVC

rbf_svm = SVC(kernel='rbf', gamma='auto')
#rbf_accuracy = cross_val_score(rbf_svm, train_X, train_y,
#                             scoring='accuracy', cv=10)

In [41]:
# mean accuracy of non-linear SVM
#np.mean(rbf_accuracy)

In [42]:
#rbf_auc = cross_val_score(rbf_svm, train_X, train_y,
#                         scoring='roc_auc', cv=10)

In [43]:
# mean AUC of non-linear SVM
#np.mean(rbf_auc)

### Model Tuning

Based on the above, choose two methods and fit a tuned model:
    - use 5-fold cross validation for model selection
    - use 10-fold cross validation for model assessment (based on appropriate performance metric)

Report estimated performance for both tuned classifiers

In [44]:
# tune your models here
# using Random Forest and Linear SVM

In [45]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score

In [46]:
from sklearn.ensemble import RandomForestClassifier

rf_param_grid = [
    {'n_estimators': [3, 10, 30], 'max_features': [2, 5, 10, 20]}
]

rf_classifier = RandomForestClassifier()
rf_tuner = GridSearchCV(rf_classifier, rf_param_grid, cv=5,
                        scoring='roc_auc', n_jobs=4)

In [47]:
tuned_rf_auc = cross_val_score(rf_tuner, train_X, train_y,
                              scoring='roc_auc', cv=10)

KeyboardInterrupt: 

In [ ]:
import numpy as np

# mean tuned random forest AUC
np.mean(tuned_rf_auc)

In [ ]:
from sklearn.svm import LinearSVC

svm_param_grid = [
    {'C': 2 ** np.linspace(-3, 5, num=5)}
]

svm_classifier = LinearSVC()
svm_tuner = GridSearchCV(svm_classifier, svm_param_grid, cv=5,
                        scoring='roc_auc', n_jobs=4, verbose=1)

In [ ]:
tuned_svm_auc = cross_val_score(svm_tuner, train_X, train_y,
                               scoring='roc_auc', cv=10, verbose=1)

In [18]:
np.mean(tuned_svm_auc)

0.66791707563522429

### Linear SVM with Gradient Descent

In [70]:
import numpy as np
import math

# initialize model parameters w and b
# intializing to 0 is not a good idea
# it should be a random vector see np.random.randn
# YOU NEED TO IMPLEMENT THIS
def _initialize_parameters(nfeatures):
    sd = math.sqrt( 0.5 * nfeatures)
    w = np.random.randn(nfeatures) / sd 
    b = np.random.randn(1) / sd
    return w, b

Compute the signed distances for observations $X$, given weights $w$ and offset (bias) $b$

$$
f = Xw + b
$$

In [71]:
# compute the signed distances
# based on current model estimates
# w and b
# YOU NEED TO IMPLEMENT THIS
def _get_signed_distances(X, w, b):
    f = np.dot(X, w) + b
    return f

Objective function for linear SVM

$$
L(w,b) = \frac{1}{n} \sum_{i=1}^n (1-y_i f_i)_+ + \frac{\lambda}{2} \| w \|^2
$$

In [72]:
# this is a vectorized version of positive_part operation
# we can use this for hinge loss as post_part(1.0 - y*f)
pos_part = np.vectorize(lambda u: u if u > 0. else 0.)

# compute the value of the linear SVM objective function
# given current signed distances, and parameter vector w
def _get_objective(f, y, w, lam):
    nobs = y.shape[0]
    loss = np.sum(pos_part(1.0 - y*f)) / nobs
    penalty = 0.5 * lam * np.dot(w,w)
    return loss + penalty

Gradients of the objective function

$$
\nabla_w L = \frac{1}{n} \sum_{i=1}^n t_i y_i x_i + \lambda w
$$

$$
\frac{\partial L}{\partial b} = \frac{1}{n} \sum_{i=1}^n t_i y_i
$$

where 

$$
t_i =
\begin{cases}
-1 & \mathrm{if} \; y_if_i < 1 \\
0 & \mathrm{o.w.}
\end{cases}
$$

In [74]:
# compute gradients with respect to w and b
# YOU NEEED TO IMPLEMENT THIS
subgrad = np.vectorize(lambda yf: -1. if  yf < 1. else 0.)

def _get_gradients(f, X, y, w, b, lam):
    nobs = X.shape[0]
    yf = y * f
    t = subgrad(yf)
    ty = t * y
    
    gw = np.sum(np.multiply(X.T, ty).T, axis=0) / nobs
    gw += lam * w
    gb = np.sum(ty) / nobs
    return gw, gb

In [78]:
# fit an SVM using gradient descent
# X: matrix of feature values
# y: labels (-1 or 1)
# n_iter: numer of iterations
# eta: learning rate
# tol: if the norm of the gradient becomes less than this, stop iterations
# verbose: print iteration information if > 0
def fit_svm(X, y, lam, n_iter=1000, eta=.5, tol=1e-10, verbose=0):
    nexamples, nfeatures = X.shape
    print_size = round(n_iter / 10.)
    
    w, b = _initialize_parameters(nfeatures)
    
    for k in range(n_iter):
        f = _get_signed_distances(X, w, b)
        
        # print information and 
        # update the learning rate
        if k % print_size == 0:
            obj = _get_objective(f, y, w, lam)
            penalty = lam * np.dot(w,w)
            loss = obj - penalty
            grad_size = np.sum(gw**2) if k > 0 else np.inf
            
            if verbose > 0:
                print("it: %d, obj %.2f, loss %.2f, g: %.2f" % (k, obj, loss, grad_size))
                
            # check stopping condition
            if grad_size < tol:
                break
                
            # update the learning rate
            eta = 0.5 * eta
        
        gw, gb = _get_gradients(f, X, y, w, b, lam)
        
        # update model parameters
        w = w - eta * gw
        b = b - eta * b
    return w, b

In [79]:
nobs = train_X.shape[0]
w,b = fit_svm(train_X, train_y, lam=1.0 , n_iter=1000)

Use these functions within scikit learn infrastructure so we can tune and compute cross validation score

In [82]:
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.utils.validation import check_X_y, check_array, check_is_fitted

class GDLinearSVM(BaseEstimator, ClassifierMixin):
    def __init__(self, lam=1.0, n_iter=1000, eta=.5, tol=1e-10, verbose=0):
        self.lam = lam
        self.n_iter = n_iter
        self.eta = eta
        self.tol = tol
        self.verbose = verbose
        
    def fit(self, X, y):
        X, y = check_X_y(X, y)
        
        self.w_, self.b_ = fit_svm(X, y, lam=self.lam, n_iter=self.n_iter,
                                  eta = self.eta, tol=self.tol, verbose=self.verbose)
        return self
    
    def decision_function(self, X):
        check_is_fitted(self, ['w_', 'b_'])
        X = check_array(X)
        
        return _get_signed_distances(X, self.w_, self.b_)
    
    def predict(self, X):
        f = self.decision_function(X)
        return np.sign(f)


In [89]:
from sklearn.model_selection import GridSearchCV

gdsvm = GDLinearSVM()
svm_param_grid = [
    {'lam': 2 ** np.linspace(-3, 5, num=5)}
]

gdsvm_tuner = GridSearchCV(gdsvm, svm_param_grid, cv=5, n_jobs=4, verbose=1, scoring='roc_auc')

In [90]:
gdsvm_tuner.fit(train_X,train_y)

Fitting 5 folds for each of 5 candidates, totalling 25 fits


[Parallel(n_jobs=4)]: Done  25 out of  25 | elapsed:  5.3min finished


GridSearchCV(cv=5, error_score='raise',
       estimator=GDLinearSVM(eta=0.5, lam=1.0, n_iter=1000, tol=1e-10, verbose=0),
       fit_params={}, iid=True, n_jobs=4,
       param_grid=[{'lam': array([  0.125,   0.5  ,   2.   ,   8.   ,  32.   ])}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='roc_auc', verbose=1)